# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
file='../python-api-challenge/WeatherPy.csv' 
WeatherPy_df=pd.read_csv(file)

WeatherPy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,80.64,61,99.0,4.29,US,1.659308e+09
1,Mahébourg,-20.4081,57.7000,70.25,73,75.0,11.50,MU,1.659308e+09
2,Bluff,-46.6000,168.3333,46.80,76,51.0,15.17,NZ,1.659308e+09
3,Provideniya,64.3833,-173.3000,48.36,81,0.0,4.47,RU,1.659308e+09
4,Lebu,-37.6167,-73.6500,47.01,83,95.0,5.30,CL,1.659308e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:

max_hum=WeatherPy_df['Humidity'].max()



gmaps.configure(api_key=g_key)

WeatherPy_df = WeatherPy_df.dropna()
humidity = WeatherPy_df["Humidity"].astype(float)

locations = WeatherPy_df[["Lat", "Lng"]].astype(float)

fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=max_hum,
                               point_radius = 4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
filtered_weather_df=WeatherPy_df[(WeatherPy_df['Max Temp'] < 80) &
                             (WeatherPy_df['Max Temp'] > 70) &
                             (WeatherPy_df['Wind Speed'] < 10) &
                             (WeatherPy_df['Cloudiness'] == 0)]

filtered_weather_df=filtered_weather_df.dropna() # drop any rows will null values

filtered_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,Termez,37.2242,67.2783,77.09,29,0.0,9.22,UZ,1.659308e+09
224,Kropotkin,45.4375,40.5756,75.90,47,0.0,9.13,RU,1.659308e+09
235,Vila,42.0304,-8.1588,71.71,58,0.0,3.85,PT,1.659308e+09
242,Old Town,50.0870,14.4202,73.47,64,0.0,5.75,CZ,1.659308e+09
253,Natal,-5.7950,-35.2094,79.02,89,0.0,9.22,BR,1.659308e+09
271,Barbate,36.1924,-5.9219,75.07,90,0.0,7.09,ES,1.659308e+09
301,Shiyan,32.6475,110.7781,75.97,77,0.0,0.16,CN,1.659308e+09
360,Franklin,39.9667,-83.0166,79.93,62,0.0,5.75,US,1.659308e+09
364,Le Mans,48.0000,0.2000,71.53,64,0.0,3.44,FR,1.659308e+09
409,Alto Araguaia,-17.3147,-53.2153,70.79,32,0.0,1.92,BR,1.659308e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = filtered_weather_df


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}


for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

  
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print('-------------------------------')
    

Retrieving Results for Index 103: Termez.
Closest hotel is Hotel Sharq.
-------------------------------
Retrieving Results for Index 224: Kropotkin.
Closest hotel is Hotel Europe Kropotkin.
-------------------------------
Retrieving Results for Index 235: Vila.
Closest hotel is Hotel Castrum Villae by Walk Hotels.
-------------------------------
Retrieving Results for Index 242: Old Town.
Closest hotel is Botanique Hotel Prague.
-------------------------------
Retrieving Results for Index 253: Natal.
Closest hotel is Villa Park Hotel | Natal.
-------------------------------
Retrieving Results for Index 271: Barbate.
Closest hotel is Hotel Adiafa.
-------------------------------
Retrieving Results for Index 301: Shiyan.
Closest hotel is Fenghuang Garden Holiday Hotel.
-------------------------------
Retrieving Results for Index 360: Franklin.
Closest hotel is Courtyard by Marriott Columbus Downtown.
-------------------------------
Retrieving Results for Index 364: Le Mans.
Closest hotel

In [15]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
103,Termez,37.2242,67.2783,77.09,29,0.0,9.22,UZ,1.659308e+09,Hotel Sharq
224,Kropotkin,45.4375,40.5756,75.90,47,0.0,9.13,RU,1.659308e+09,Hotel Europe Kropotkin
235,Vila,42.0304,-8.1588,71.71,58,0.0,3.85,PT,1.659308e+09,Hotel Castrum Villae by Walk Hotels
242,Old Town,50.0870,14.4202,73.47,64,0.0,5.75,CZ,1.659308e+09,Botanique Hotel Prague
253,Natal,-5.7950,-35.2094,79.02,89,0.0,9.22,BR,1.659308e+09,Villa Park Hotel | Natal
271,Barbate,36.1924,-5.9219,75.07,90,0.0,7.09,ES,1.659308e+09,Hotel Adiafa
301,Shiyan,32.6475,110.7781,75.97,77,0.0,0.16,CN,1.659308e+09,Fenghuang Garden Holiday Hotel
360,Franklin,39.9667,-83.0166,79.93,62,0.0,5.75,US,1.659308e+09,Courtyard by Marriott Columbus Downtown
364,Le Mans,48.0000,0.2000,71.53,64,0.0,3.44,FR,1.659308e+09,ibis Styles Le Mans Centre Gare
409,Alto Araguaia,-17.3147,-53.2153,70.79,32,0.0,1.92,BR,1.659308e+09,Araguaia Palace Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))